In [1]:
import numpy as np
import matplotlib as plt
from osgeo import gdal
import pandas as pd
import os



In [33]:
wavelen_dict={6:43300.0,10:28200.0,18:16000.0,36:8200.0,89:3400.0}

corr=pd.read_csv(r"E:23sept/LUT_OUTPUT/North_east_JUNE_1_10.csv")
# remove 23GHZ(12600um)
corr=corr[corr['wavelen_um']!=12600]
corr.head()

,Unnamed: 0,DOY,wavelen_um,Transmissivity,Upwelling_BT(K),Downwelling_BT(K)
0,0,10,3400.0,0.692064,62.6690,72.6199
1,1,10,8200.0,0.707603,156.0130,193.6840
3,3,10,16000.0,0.920671,31.4334,35.0340
4,4,10,28200.0,0.937878,21.7138,24.2554
5,5,10,43300.0,0.941592,20.8122,23.2796


In [34]:
b='AMSER-E_monthly_emissivity_Mayl200610.7'
print(len(b))
print(b[35:37])

39
10


In [35]:
LST_path=r'E:23sept/LST/june_LST/'
lst_dict={}
LST_list=[]
doy_num=151
for root, dirs, files in os.walk(LST_path):
    for name in files:
        LST_list.append(LST_path+name)
for i in range(len(LST_list)):
    lst_dict[i+1+doy_num]=LST_list[i]
   

emiss_path = r'E:23sept/Emissivity/june_emiss/'
emiss_dict={}
for root, dirs, files in os.walk(emiss_path):
    for name in files:
        if 'V.tif' in name:
            W=int(name[35:37].strip('.'))
#             emiss_list.append(emiss_path+name)
            emiss_dict[W]=emiss_path+name

In [36]:
print(lst_dict,'\n')
print(emiss_dict)

{152: 'E:23sept/LST/june_LST/MYD11C1.A2006152.006.2015292174138.hdf.tif.tif', 153: 'E:23sept/LST/june_LST/MYD11C1.A2006153.006.2015292142521.hdf.tif.tif', 154: 'E:23sept/LST/june_LST/MYD11C1.A2006154.006.2015292142544.hdf.tif.tif', 155: 'E:23sept/LST/june_LST/MYD11C1.A2006155.006.2015292142520.hdf.tif.tif', 156: 'E:23sept/LST/june_LST/MYD11C1.A2006156.006.2015292142522.hdf.tif.tif', 157: 'E:23sept/LST/june_LST/MYD11C1.A2006157.006.2015292142525.hdf.tif.tif', 158: 'E:23sept/LST/june_LST/MYD11C1.A2006158.006.2015292152053.hdf.tif.tif', 159: 'E:23sept/LST/june_LST/MYD11C1.A2006159.006.2015292174142.hdf.tif.tif', 160: 'E:23sept/LST/june_LST/MYD11C1.A2006160.006.2015292174142.hdf.tif.tif', 161: 'E:23sept/LST/june_LST/MYD11C1.A2006161.006.2015292184846.hdf.tif.tif'} 

{10: 'E:23sept/Emissivity/june_emiss/AMSER-E_monthly_emissivity_June200610.7 V.tif.tif', 18: 'E:23sept/Emissivity/june_emiss/AMSER-E_monthly_emissivity_June200618.7 V.tif.tif', 36: 'E:23sept/Emissivity/june_emiss/AMSER-E_monthl

In [37]:
LST_error_path=r'E:/23sept/LST_error/'

In [38]:
def WriteImage(outpath,image,proj,geo):
    ds=None
    driver=gdal.GetDriverByName("GTiff")
    ds=driver.Create(outpath,image.shape[1],image.shape[0], 1,gdal.GDT_Float32)
    ds.SetProjection(proj)
    ds.SetGeoTransform(geo)
    ds.GetRasterBand(1).WriteArray(image)
    return ds

In [29]:
# BT_path=r"E:23sept/Resampled_BT_clipped/april_BT//"
# for root, dirs, files in os.walk(BT_path):
#     for name in files:
#         if ('V_Res_1' in name) or ('89_0V_Res_4' in name):
#             BTfilePath=BT_path+name
#             print(name)
#             BT_DOY=int(name[30:32])+120
#             print(BT_DOY)
#             wavelen=int(name[57:59])
#             print(wavelen)

In [39]:
df=pd.DataFrame(columns=['DOY','Wavelenght','MODIS_LST','LST_corrected','LST_raw'])

In [40]:
# BT data
BT_path=r"E:23sept/Resampled_BT_clipped/june_BT//"
for root, dirs, files in os.walk(BT_path):
    for name in files:
        if ('V_Res_1' in name) or ('89_0V_Res_4' in name):
        
            BTfilePath=BT_path+name
#             print(BTfilePath)
#             BT_DOY=int(name[30:32])+90
            BT_DOY=int(name[30:32])
            wavelen=int(name[57:59])

#             print(wavelen)
            BT=gdal.Open(BTfilePath)
            proj=BT.GetProjection()
            geotrans=BT.GetGeoTransform()
            BT_raw=BT.ReadAsArray().astype("float64")
            
            new_df=corr[(corr['DOY']==BT_DOY) & (corr['wavelen_um']==wavelen_dict[wavelen])]
#             print(new_df)
            upwell=np.array(new_df['Upwelling_BT(K)'])
            downwell=np.array(new_df['Downwelling_BT(K)'])
            trans=np.array(new_df['Transmissivity'])
#             print(upwell,downwell,trans)
            emissFile=emiss_dict[wavelen]
            emiss=gdal.Open(emissFile)
            emiss=emiss.ReadAsArray().astype('float64')
            emiss[emiss<0.5]=np.nan
#             emiss[emiss>0.99999]=np.nan
            
            estimated_LST=(BT_raw-upwell-(1-emiss)*((downwell+2.725)*trans))/(emiss*trans)
            raw_BT_LST=BT_raw/emiss
            
            MOD_LSTpath=lst_dict[BT_DOY+doy_num]
            MOD_LST=gdal.Open(MOD_LSTpath)
            MOD_LST=MOD_LST.ReadAsArray().astype('float64')
            
            error=estimated_LST-MOD_LST
#             storeLST  in csv
            f=pd.DataFrame({})
            f['MODIS_LST']=(MOD_LST[44:55,58:69]).flatten()
            f['LST_corrected']=(estimated_LST[44:55,58:69]).flatten()
            f['LST_raw']=(raw_BT_LST[44:55,58:69]).flatten()
            f['DOY']=BT_DOY+doy_num
            f['Wavelenght']=wavelen
            
            df=df.append(f,ignore_index=True)
#             print(f)
            f=None            
#             outpath=LST_error_path+'LST_error_June_'+str(BT_DOY)+"_"+name[56:62]+'.tif'
#             VE=WriteImage(outpath,error,proj,geotrans)
            
#             VE=None
            
            
        

In [41]:
df.to_csv(r'E:23sept/Error_csv/North_east_June.csv')

In [11]:
len('AMSER-E_monthly_emissivity_April2006')

36

In [69]:
df

,DOY,MODIS_LST,LST_corrected,LST_raw,Wavelenght
0,121,316.333984,NaN,NaN,6.0
1,121,316.747437,NaN,NaN,6.0
2,121,318.998016,NaN,NaN,6.0
3,121,318.735046,NaN,NaN,6.0
4,121,319.401794,NaN,NaN,6.0
...,...,...,...,...,...
6045,130,NaN,297.302362,271.780621,89.0
6046,130,NaN,272.089086,254.419044,89.0
6047,130,NaN,304.438175,277.346819,89.0
6048,130,NaN,NaN,NaN,89.0


In [12]:
emiss_dict

{10: 'E:23sept/april_emiss/AMSER-E_monthly_emissivity_April200610.7 V.tif.tif',
 18: 'E:23sept/april_emiss/AMSER-E_monthly_emissivity_April200618.7 V.tif.tif',
 36: 'E:23sept/april_emiss/AMSER-E_monthly_emissivity_April200636.5 V.tif.tif',
 6: 'E:23sept/april_emiss/AMSER-E_monthly_emissivity_April20066.9  V.tif.tif',
 89: 'E:23sept/april_emiss/AMSER-E_monthly_emissivity_April200689.0 V.tif.tif'}

In [13]:
a=np.array([1,2,3])
b=np.array([2])
a-b

array([-1,  0,  1])